In [16]:
import math, subprocess
import subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import os

In [17]:
def leerInstancia(path):
    with open(path, "r") as f:
        return f.read();

In [18]:
def leerOptimo(instancia):
    with open(F"instancias/CMI_n{instancia}.out", "r") as f:
        return f.readlines()[0];

In [19]:
def correrPrograma(argumentos, instancia):
    #Leer archivo de la instancia
    instancia = leerInstancia(instancia);
    
    #crear proceso para ejecutar el codigo.
    process = subprocess.Popen(argumentos, stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)
    
    #Poner la instancia en la entrada standar.
#     process.stdin.write(instancia)
#     process.stdin.flush()
    res = process.communicate(input=instancia);
    
    #print(res[0])
    
    resultados = res[0].split(" ");
    impacto = int(resultados[0]);
    tiempo = float(resultados[1]);
    
#     print(F"res: {res}");
#     print(F"resultados: {resultados}");
#     print(F"impacto: {impacto}");
    
    #Correr experimento
    exit_code = process.wait()
    
    #Verificar que no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el programa: {programa} con la instancia {instancia}")
    
    #Leer la salida de STDERR con los tiempos de ejecucion de cada metodo
#     tiempo_de_ejecucion = float(process.stderr.read());
    
    process.stdin.close(); 
    process.stdout.close();
    process.stderr.close();
    
    return (impacto, tiempo)

In [20]:
def correrExperimentos(instancia, argumentos, parametro, valores_parametro, cant_repeticiones, output):   
    filas = [];
    columnas = [parametro, "optimo", "impacto", "tiempo(ms)"];
    
    parametro_i = 0;
    print(F"Instancia: {instancia}");
    for _argumentos in argumentos:
        # Voy mostrando la instancia se esta ejecutando.
        clear_output(wait=True);
        print(F"{parametro}: {valores_parametro[parametro_i]}"); 

        # Ejecutamos el experimento cant_repeticiones veces y obtenemos la media.
        impactos = [];
        tiempos = [];
        for i in range(0, cant_repeticiones):
            print(_argumentos);
            resultado = correrPrograma(_argumentos, F"instancias/CMI_n{instancia}.in");
            impactos.append(resultado[0]);
            tiempos.append(resultado[1]);
            
        impacto = np.median(impactos);
        tiempo = np.median(tiempos);
        
        filas.append([
            valores_parametro[parametro_i],
            leerOptimo(instancia),
            impacto,
            tiempo
        ]);
        parametro_i = parametro_i + 1;

    # Si no existe la carpeta la creo
    exp_folder = "experimentos/";
    if os.path.exists(F"{exp_folder}") == False:
        os.mkdir(F"{exp_folder}")
    df_resultado = pd.DataFrame(filas, columns=columnas);
    df_resultado.to_csv(F"{exp_folder}{output}", index=False, header=True);

In [63]:
def correrExperimentos2(instancias, argumentos, cant_repeticiones, output):   
    filas = [];
    columnas = ["instancia", "optimo", "impacto", "tiempo(ms)"];
    
    for instancia in instancias:
        print(F"Instancia {instancia}");
        
        # Ejecutamos el experimento cant_repeticiones veces y obtenemos la media.
        impactos = [];
        tiempos = [];
        for i in range(0, cant_repeticiones):
            resultado = correrPrograma(argumentos, F"instancias/CMI_n{instancia}.in");
            impactos.append(resultado[0]);
            tiempos.append(resultado[1]);
            
        impacto = np.median(impactos);
        tiempo = np.median(tiempos);
        
        filas.append([
            instancia,
            leerOptimo(instancia),
            impacto,
            tiempo
        ]);

    # Si no existe la carpeta la creo
    exp_folder = "experimentos/";
    if os.path.exists(F"{exp_folder}") == False:
        os.mkdir(F"{exp_folder}")
    df_resultado = pd.DataFrame(filas, columns=columnas);
    df_resultado.to_csv(F"{exp_folder}{output}", index=False, header=True);

#### Algoritmos Tabú: comparación de parámetros ####

In [113]:
instancias = [6, 10, 14, 20, 24, 28];
algoritmos = ["TABU3", "TABU4"];

# Parametros
algoritmos_base = ["H1", "H2"];
iteraciones = [str(i) for i in range(1, 1000, 20)];
tam_memoria = [str(i) for i in range(0, 100, 2)];
vecindades = ["0", "10", "20", "30", "40", "50", "60", "70", "80", "90", "100"];

repeticiones = 3;

for algoritmo in algoritmos:
    for instancia in instancias:
        print(f"Instancia: {instancia}")
        
        _iteraciones = "30" if algoritmo == "TABU3" else "150";
        vecindad = "20" if algoritmo == "TABU3" else "70";
        _memoria = "100" if algoritmo == "TABU3" else "100";
        
        # Parámetro 1: Iteraciones
#         argumentos = [];
#         for iteracion in iteraciones:
#             argumentos.append(["./PCMI", algoritmo, "H2", iteracion, "10", "10"]);
#         correrExperimentos(instancia, argumentos, "iteraciones", iteraciones, repeticiones, F"{algoritmo}_n{instancia}_iteraciones_1_1000.csv");
        
        # Parámetro 1 (final): Iteraciones
#         argumentos = [];
#         for iteracion in iteraciones:
#             argumentos.append(["./PCMI", algoritmo, "H2", iteracion, _memoria, vecindad]);
#         correrExperimentos(instancia, argumentos, "iteraciones", iteraciones, repeticiones, F"{algoritmo}_n{instancia}_iteraciones_1_1000_final.csv");

# #         # Parámetro 2: Vecindad
#         argumentos = [];
#         for vecindad in vecindades:
#             argumentos.append(["./PCMI", algoritmo, "H2", _iteraciones, "10", vecindad]);
#         correrExperimentos(instancia, argumentos, "vecindad", vecindades, repeticiones, F"{algoritmo}_n{instancia}_vecindad.csv");

# # Parámetro 3: Memoria
        argumentos = [];
        for memoria in tam_memoria:
            argumentos.append(["./PCMI", algoritmo, "H2", _iteraciones, memoria, vecindad]);
        correrExperimentos(instancia, argumentos, "memoria", tam_memoria, repeticiones, F"{algoritmo}_n{instancia}_memoria.csv");

memoria: 98
['./PCMI', 'TABU4', 'H2', '150', '98', '70']
['./PCMI', 'TABU4', 'H2', '150', '98', '70']
['./PCMI', 'TABU4', 'H2', '150', '98', '70']


30


#### Comparación de Heurísticas Golosas ####

In [37]:
instancias = [6, 10, 14, 20, 24, 28];
_instancias = [6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30];

algoritmos_base = ["H1", "H2"];
repeticiones = 3;
for algoritmo in algoritmos_base:
    argumentos = ["./PCMI", algoritmo, "H1", "999", "999", "999"];
    correrExperimentos2(instancias, argumentos, repeticiones, F"heuristica_{algoritmo}.csv");

Instancia 6
Instancia 10
Instancia 14
Instancia 20
Instancia 24
Instancia 28
Instancia 6
Instancia 10
Instancia 14
Instancia 20
Instancia 24
Instancia 28


#### Parámetro Inicial: heurística ####

In [75]:
instancias = [6, 10, 14, 20, 24, 28];
_instancias = [6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30];

tabues = ["TABU3", "TABU4"];
algoritmos_base = ["H1", "H2"];

repeticiones = 3;
for tabu in tabues:
    for algoritmo in algoritmos_base:
        argumentos = ["./PCMI", tabu, algoritmo, "10", "10", "10"];
        correrExperimentos2(instancias, argumentos, repeticiones, F"{tabu}_{algoritmo}_heuristica.csv");

Instancia 6
Instancia 10
Instancia 14
Instancia 20
Instancia 24
Instancia 28
Instancia 6
Instancia 10
Instancia 14
Instancia 20
Instancia 24
Instancia 28
Instancia 6
Instancia 10
Instancia 14
Instancia 20
Instancia 24
Instancia 28


KeyboardInterrupt: 

#### Experimento 3: Configuración óptima para instancias complementarias ####

In [116]:
instancias = [6, 10, 14, 20, 24, 28];
complementarias = [8, 12, 16, 18, 22, 26, 30];
_instancias = [6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30];

tabues = ["TABU3", "TABU4"];

repeticiones = 3;

for tabu in tabues:
    algoritmo_inicial = "H2";
    _iteraciones = "350" if tabu == "TABU3" else "150";
    vecindad = "20" if tabu == "TABU3" else "70";
    _memoria = "44" if tabu == "TABU3" else "1";
    
    argumentos = ["./PCMI", tabu, algoritmo_inicial, _iteraciones, _memoria, vecindad];
    correrExperimentos2(instancias, argumentos, repeticiones, F"{tabu}_experimento3.csv");

Instancia 6
Instancia 10
Instancia 14
Instancia 20
Instancia 24
Instancia 28
Instancia 6
Instancia 10
Instancia 14
Instancia 20
Instancia 24
Instancia 28
